# Installing necessary packages, can ask the users to restart the notebook

In [4]:
# !sudo apt-get install -y libjpeg-dev zlib1g-dev  # Do this, if pillow fails

In [ ]:
%pip install scikit-image PyMuPDF python-docx opencv-python scipy torch torchvision==0.2.1 pillow==8.3.1 tensorflow==2.15 gdown

You will be prompted to Restart the session. Continue running the cells after restarting

# Importing the necessary modules

In [2]:
# Data Pre-processing
import fitz
import cv2
import csv
import os
from docx import Document
import string
import requests
import pandas as pd
import numpy as np
from PIL import Image, ImageOps
from IPython.display import clear_output as cls

# Data
from glob import glob
from tqdm import tqdm
import tensorflow.data as tfd

# Data Visualization
import matplotlib.pyplot as plt

# Model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.utils import plot_model

2025-03-22 18:57:53.655177: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-22 18:57:53.682192: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-22 18:57:53.682221: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-22 18:57:53.683109: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-22 18:57:53.687842: I tensorflow/core/platform/cpu_feature_guar

# Downloading the dataset

In [3]:
# Downloading the Dataset on collab/local notebook

# Virtuosa ( Dataset 1)
!gdown 'http://drive.google.com/uc?id=10NX_UbV2HMbPEO2fvKYAIXOOOec0g38g'  # Downloading link for Ancient Text
!gdown 'http://drive.google.com/uc?id=1YTaqNoZCYP74AuQxlyJsiQLhcoc8DNSv'  # Downloading link for Ground Truth Text

# Perfecto ( Dataset 2)
!gdown 'http://drive.google.com/uc?id=1x6FS3z4WhsHS7s38a2oSH8JnLQ_u_f21'  # Downloading link for Ancient Text
!gdown 'http://drive.google.com/uc?id=1YqQ04ZQR4xdjKlQS9xL9zkkWYx_Ppxwa'  # Downloading link for Ground Truth Text

!gdown 'http://drive.google.com/uc?id=1r7TjJ9RjNZHxAzKhd4uOaRWanrQXIqw8' # Downloading utils.py from cloud

Downloading...
From: http://drive.google.com/uc?id=10NX_UbV2HMbPEO2fvKYAIXOOOec0g38g
To: /home/shashank/Desktop/GSOC'25/HumanAI_Task_submission_1/Padilla - Nobleza virtuosa_testExtract.pdf
100%|██████████████████████████████████████| 2.58M/2.58M [00:00<00:00, 6.39MB/s]
Downloading...
From: http://drive.google.com/uc?id=1YTaqNoZCYP74AuQxlyJsiQLhcoc8DNSv
To: /home/shashank/Desktop/GSOC'25/HumanAI_Task_submission_1/Padilla - 1 Nobleza virtuosa_testTranscription.docx
100%|██████████████████████████████████████| 29.1k/29.1k [00:00<00:00, 1.73MB/s]
Downloading...
From: http://drive.google.com/uc?id=1x6FS3z4WhsHS7s38a2oSH8JnLQ_u_f21
To: /home/shashank/Desktop/GSOC'25/HumanAI_Task_submission_1/Padilla - 2 Noble perfecto_Extract.pdf
100%|██████████████████████████████████████| 3.95M/3.95M [00:00<00:00, 7.28MB/s]
Downloading...
From: http://drive.google.com/uc?id=1YqQ04ZQR4xdjKlQS9xL9zkkWYx_Ppxwa
To: /home/shashank/Desktop/GSOC'25/HumanAI_Task_submission_1/Padilla - 2 Noble perfecto_Transcriptio

In [4]:
!ls

 Dataset_Generation.ipynb
 Model.ipynb
'Padilla - 1 Nobleza virtuosa_testTranscription.docx'
'Padilla - 2 Noble perfecto_Extract.pdf'
'Padilla - 2 Noble perfecto_Transcription.docx'
'Padilla - Nobleza virtuosa_testExtract.pdf'
 README.md
 utils.py


# Pre-processing the dataset

### Converting PDF to images

In [5]:
from utils import pdf_to_images

pdf_path1 = "./Padilla - Nobleza virtuosa_testExtract.pdf"  # Path to the PDF file
unproc_images_folder_1 = "./preprocessing/imgsUnProcessed1"  # Output folder to save the images
if not os.path.exists(unproc_images_folder_1):
    os.makedirs(unproc_images_folder_1)
pdf_to_images(pdf_path1, unproc_images_folder_1)


pdf_path2 = "./Padilla - 2 Noble perfecto_Extract.pdf"  # Path to the PDF file
unproc_images_folder_2 = "./preprocessing/imgsUnProcessed2"  # Output folder to save the images
if not os.path.exists(unproc_images_folder_2):
    os.makedirs(unproc_images_folder_2)
pdf_to_images(pdf_path2, unproc_images_folder_2)

<img src="https://raw.githubusercontent.com/ML4SCI/DeepLearnHackathon/main/NLPRenaissanceChallenge/images/imageOriginal.png">

### Splitting two sided scanned images into individual pages.

In [6]:
#Segregating 2-sided pages into individual pages
from utils import process_images

unproc_images_folder_1 = "./preprocessing/imgsUnProcessed1"
proc_images_folder_1 = "./preprocessing/imgsForAllPages1"
if not os.path.exists(proc_images_folder_1):
    os.makedirs(proc_images_folder_1)
process_images(unproc_images_folder_1, proc_images_folder_1)


unproc_images_folder_2 = "./preprocessing/imgsUnProcessed2"
proc_images_folder_2 = "./preprocessing/imgsForAllPages2"
if not os.path.exists(proc_images_folder_2):
    os.makedirs(proc_images_folder_2)
process_images(unproc_images_folder_2, proc_images_folder_2)

print("Image processing complete!")

Image processing complete!


In [7]:
!git clone https://github.com/Shashankss1205/CRAFT_Model

Cloning into 'CRAFT_Model'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 28 (delta 0), reused 3 (delta 0), pack-reused 23 (from 1)
Receiving objects: 100% (28/28), 73.69 MiB | 3.54 MiB/s, done.


# Text Detection
### Extracting words from a scanned text page image can be achieved using any model of your choice. We are using the [CRAFT Model](https://github.com/clovaai/CRAFT-pytorch) for the same. (This will take 1-2 mins to process the entire model)

In [8]:
#It generally takes about ~3-4 mins
!python3 CRAFT_Model/CRAFT/BoundBoxFunc/test.py --cuda 1 --result_folder='./preprocessing/BoundBoxApplied1/' --test_folder="./preprocessing/imgsForAllPages1" --trained_model='CRAFT_Model/CRAFT/BoundBoxFunc/weights/craft_mlt_25k.pth'

Loading weights from checkpoint (CRAFT_Model/CRAFT/BoundBoxFunc/weights/craft_mlt_25k.pth)
elapsed time : 18.305070877075195simgsForAllPages1/image_15.png


In [ ]:
#It generally takes about ~3-4 mins
!python3 CRAFT_Model/CRAFT/BoundBoxFunc/test.py --cuda 1 --result_folder='./preprocessing/BoundBoxApplied2/' --test_folder="./preprocessing/imgsForAllPages2" --trained_model='CRAFT_Model/CRAFT/BoundBoxFunc/weights/craft_mlt_25k.pth'

Loading weights from checkpoint (CRAFT_Model/CRAFT/BoundBoxFunc/weights/craft_mlt_25k.pth)
elapsed time : 18.559587001800537simgsForAllPages2/image_15.png


### The output of this model provides coordinates of the polygon enclosing the word. Using these coordinates one can draw a bounding box and crop word images.

In [10]:
#Sorting the BB based on the Spanish writing style
from utils import sort_bounding_boxes

bound_box_applied1 = './preprocessing/BoundBoxApplied1/'
bound_box_sorted1 = "./preprocessing/BoundBoxSorted1"
if not os.path.exists(bound_box_sorted1):
    os.makedirs(bound_box_sorted1)
sort_bounding_boxes(bound_box_applied1, bound_box_sorted1)

bound_box_applied2 = './preprocessing/BoundBoxApplied2/'
bound_box_sorted2 = "./preprocessing/BoundBoxSorted2"
if not os.path.exists(bound_box_sorted2):
    os.makedirs(bound_box_sorted2)
sort_bounding_boxes(bound_box_applied2, bound_box_sorted2)

<img src="https://raw.githubusercontent.com/ML4SCI/DeepLearnHackathon/main/NLPRenaissanceChallenge/images/imageCRAFT.jpg">

### Extracting all the words from .docx file containing Transcription (True lables).

In [11]:
#ground truth to text code
from utils import save_pages_to_text

docx_file1 = "./Padilla - 1 Nobleza virtuosa_testTranscription.docx"
grnd_truth1 = "./preprocessing/all_text1.txt"     # File where the complete processed text will be saved
save_pages_to_text(docx_file1, grnd_truth1)

docx_file2 = "./Padilla - 2 Noble perfecto_Transcription.docx"
grnd_truth2 = "./preprocessing/all_text2.txt"     # File where the complete processed text will be saved
save_pages_to_text(docx_file2, grnd_truth2)

### Since we have processed our text book into individual pages, we need to split the entire transcription based on text pages.

In [12]:
#splitting text files, based on the text
from utils import process_textfiles

grnd_truth1 = "./preprocessing/all_text1.txt"
bound_box_sorted1 = './preprocessing/BoundBoxSorted1'
proc_grnd_truth1 = "./preprocessing/textSplitted1"
TEST_SIZE=6
if not os.path.exists(proc_grnd_truth1):
    os.makedirs(proc_grnd_truth1)
process_textfiles(grnd_truth1, bound_box_sorted1, proc_grnd_truth1, TEST_SIZE)


grnd_truth2 = "./preprocessing/all_text2.txt" 
bound_box_sorted2 = './preprocessing/BoundBoxSorted2'
proc_grnd_truth2 = "./preprocessing/textSplitted2"
TEST_SIZE=0
if not os.path.exists(proc_grnd_truth2):
    os.makedirs(proc_grnd_truth2)
process_textfiles(grnd_truth2, bound_box_sorted2, proc_grnd_truth2, TEST_SIZE)
print("Text splitting complete!")

31 files in the folder
31 files in the folder
Text splitting complete!


### Extracts and saves bounding boxes from images using text data for filenames, skipping the last 6 images for testing.

In [13]:
from utils import apply_extraction_to_folder_for_train, count_files_in_folder

proc_images_folder_1 = './preprocessing/imgsForAllPages1'
bound_box_sorted1 = './preprocessing/BoundBoxSorted1'
proc_grnd_truth1 = './preprocessing/textSplitted1'
training_data1 = './traning_data1'
test_size=6
train_size = count_files_in_folder(proc_images_folder_1, ['.png', '.jpeg', '.jpg'])- test_size
print("Training pages " +  str(train_size))
if not os.path.exists(training_data1):
    os.makedirs(training_data1)
apply_extraction_to_folder_for_train(proc_images_folder_1, bound_box_sorted1, proc_grnd_truth1, training_data1, train_size)


proc_images_folder_2 = './preprocessing/imgsForAllPages2'
bound_box_sorted2 = './preprocessing/BoundBoxSorted2'
proc_grnd_truth2 = './preprocessing/textSplitted2'
training_data2 = './traning_data2'
test_size = 0
train_size = count_files_in_folder(proc_images_folder_1, ['.png', '.jpeg', '.jpg'])- test_size
print("Training pages " +  str(train_size))
if not os.path.exists(training_data2):
    os.makedirs(training_data2)
apply_extraction_to_folder_for_train(proc_images_folder_2, bound_box_sorted2, proc_grnd_truth2, training_data2, train_size) # better to send no. of pages given in transcription

Training pages 25
Training pages 31
